In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# Toleration: how many Nans are accepted for a feature? or rate?
# Special Nans: was Nan or not
# PSI, WOE, IVs
# Select features according to weights: pretrain with linear model
# PCA or not


In [3]:
tolerance_up = 0.8
tolerance_down = 0.3



In [19]:
train_features = pd.read_csv("./data/train/feature.csv")
train_labels = pd.read_csv("./data/train/label.csv")
train = pd.merge(train_features, train_labels)


features = train.drop(columns=['APPLICATION_ID', 'APPLICATION_DATE', 'DEFAULT_LABEL'])
nan_count = features.isna().sum()
data_preview = pd.DataFrame({'Variables': nan_count.index, 'Nan Count': nan_count.values, 'Nan Rate': nan_count.values / len(train)})

# pd.set_option("display.max_columns",None)
# pd.set_option("display.max_rows",None)
data_preview.sort_values("Variables",inplace=True)
# data_preview.loc[data_preview['Nan Rate'] < 0.8]



In [20]:
train

,APPLICATION_ID,v000,v001,v002,v003,v004,v005,v006,v007,v008,...,v992,v993,v994,v995,v996,v997,v998,v999,APPLICATION_DATE,DEFAULT_LABEL
0,44c4e0d49d5bf94efa446e03ecf84ba5,NaN,NaN,NaN,5.528423,NaN,NaN,10.0,0.0,0.016305,...,NaN,NaN,NaN,NaN,41.0,0.0,NaN,0.0,2021-09-14,0
1,993ee0eed4ad96f4f5d6dc30d56bdf94,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,NaN,...,NaN,NaN,NaN,NaN,8.0,0.0,NaN,0.0,2021-07-06,0
2,a9815449c0a385ee0d53305b8cca95c6,NaN,NaN,NaN,5.062508,0.000000,NaN,7.0,0.0,1.039721,...,NaN,NaN,NaN,NaN,22.0,0.0,NaN,0.0,2021-06-10,0
3,4f9dd0d609cee4360e110d56cc35afd9,0.0,NaN,NaN,2.575101,0.444610,NaN,NaN,NaN,0.980110,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,2021-07-04,0
4,904de5e19a5ea36e8f4b9f4cf5bdd953,0.0,NaN,0.000473,5.528423,0.000000,NaN,1.0,0.0,0.950612,...,NaN,0.079084,0.415888,NaN,76.0,0.0,3.894232,0.0,2021-06-23,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73495,f40ba3404c98f365ca94a5494db00543,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,2021-08-09,0
73496,9a3ed4e399d51c56568499f8f804abbf,0.0,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,...,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,2021-06-26,0
73497,4b64b9f949949fd609008c58c19a86f4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,2021-05-23,0
73498,8dbdd0a1dc4df6983cb58e940a449fa9,NaN,NaN,0.000931,4.564210,0.288722,NaN,1.0,0.0,0.303087,...,NaN,0.087199,0.415888,0.032204,25.0,0.0,NaN,0.0,2021-05-21,0


In [21]:
test_features = pd.read_csv("./data/test/feature.csv")
test_labels = pd.read_csv("./data/test/label.csv")
test = pd.merge(test_features, test_labels)


In [22]:
# Delete all features whose nan rate >= 0.8
deleted = 0
all_len = len(train)
for col in train.columns:
    if 'v' not in col:
        continue
    else:
        if train[col].isna().sum() >= 0.8 * all_len:
            train.drop(col, axis=1, inplace=True)
            deleted += 1
deleted
    

321

In [23]:
train.head()

,APPLICATION_ID,v000,v003,v004,v006,v007,v008,v009,v010,v011,...,v984,v989,v990,v991,v995,v996,v997,v999,APPLICATION_DATE,DEFAULT_LABEL
0,44c4e0d49d5bf94efa446e03ecf84ba5,NaN,5.528423,NaN,10.0,0.0,0.016305,NaN,NaN,NaN,...,-0.488183,0.0,2.0,0.000000,NaN,41.0,0.0,0.0,2021-09-14,0
1,993ee0eed4ad96f4f5d6dc30d56bdf94,NaN,NaN,NaN,0.0,3.0,NaN,NaN,NaN,NaN,...,-0.648454,0.0,0.0,NaN,NaN,8.0,0.0,0.0,2021-07-06,0
2,a9815449c0a385ee0d53305b8cca95c6,NaN,5.062508,0.00000,7.0,0.0,1.039721,NaN,0.901091,NaN,...,-0.109459,0.0,0.0,0.166667,NaN,22.0,0.0,0.0,2021-06-10,0
3,4f9dd0d609cee4360e110d56cc35afd9,0.0,2.575101,0.44461,NaN,NaN,0.980110,NaN,2.329287,0.0,...,-0.790820,NaN,NaN,0.250000,NaN,NaN,0.0,0.0,2021-07-04,0
4,904de5e19a5ea36e8f4b9f4cf5bdd953,0.0,5.528423,0.00000,1.0,0.0,0.950612,0.0,0.901091,0.0,...,-0.278792,0.0,12.0,NaN,NaN,76.0,0.0,0.0,2021-06-23,0


In [8]:
s1 = pd.Series([2,3,3,5], index=['1', '2', '3', '4'])
s1[s1[s1.values == 3].index] += 0.5
s1

1    2.0
2    3.5
3    3.5
4    5.0
dtype: float64

In [24]:
# Computing WOE, IV for further selection
from sklearn import preprocessing
from collections import OrderedDict
import warnings; warnings.filterwarnings("ignore")


X_train = pd.read_csv("./data/train/feature.csv")
y_train = pd.read_csv("./data/train/label.csv")
X_test  = pd.read_csv("./data/test/feature.csv")
y_test  = pd.read_csv("./data/test/label.csv")

v_names = X_train.columns[1:]
df_train = pd.merge(X_train, y_train, on= 'APPLICATION_ID')
df_test = pd.merge(X_test, y_test, on= 'APPLICATION_ID')


label = "DEFAULT_LABEL"
n_bins =  5
B_total = df_train[label].sum()           # default, y = 1
G_total = len(df_train[label]) - B_total  # not default, y = 0
B_total_test = df_test[label].sum() # default, y = 1
G_total_test = len(df_test[label]) - B_total_test  # not default, y = 0


diverse_threshold = 30
WOE_list = []
woe_is = []
iv_is = []


for v_name in v_names:
#     print("Processing on", v_name)
    data_type = 'discrete'
    df_v = df_train[[v_name, label]]
    df_v_na = df_v[df_v.isnull().values==True]
    B_na = df_v_na[label].sum()
    G_na = len(df_v_na[label]) - B_na
#     print(B_na, G_na)
    
#     print(df_v)
#     print(df_v_na[label].sum())
#     print(df_v_na[label].value_counts())
#     break
    
    df_v.dropna(inplace = True)
    df_v_test = df_test[[v_name, label]]
    df_v_test.dropna(inplace=True)
    
    KBD = preprocessing.KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
    
    
    if len(df_v[v_name].unique()) > diverse_threshold:
        # continuous 离散化
        # https://scikit-learn.org/stable/modules/preprocessing.html#discretization
        data_type = 'continous'
        data = KBD.fit_transform(df_v[[v_name]])
        df_v[v_name] = data  # 离散化
        data = KBD.transform(df_v_test[[v_name]])
        df_v_test[v_name] = data  # 离散化
        
    df_data = pd.pivot_table(df_v, index = v_name,columns=label, values=label, aggfunc=len, fill_value=0)
#     print(type(df_data[0]))
#     for i in range(len(df_data[0])):
    print(df_data[0][df_data[0].values == 0].index)
    df_data[0][df_data[0][df_data[0].values == 0].index] += 0.5
    df_data[1][df_data[0][df_data[0].values == 0].index] += 0.5
#         df_data[0][i] = df_data[0][i] + 0.5
#     print(df_data[0])
#     break
#     print(df_data[1])
    df_data[0] = df_data[0] / G_total   # good y=0
    df_data[1] = df_data[1] / B_total   # bad  y=1
    ratio = np.log(df_data[0] / df_data[1])
    ratio.replace([np.inf, -np.inf], 0, inplace=True)
#     print(np.array(df_data[1]))


    # save
#     print(ratio)
#     print(np.array(ratio))
#     break

    woe_i = np.array(ratio)
    if B_na != 0 or G_na != 0:
        np.append(woe_i, math.log(math.e, B_na / B_total) - math.log(math.e, G_na / G_total))   # Nan group
    woe_is.append(woe_i)
    iv_i = []
    bad_good = np.array(df_data)
    for kk in range(len(bad_good)):
        iv_i.append((bad_good[kk][0] - bad_good[kk][1]) * woe_i[kk])
    iv_i.append((B_na / B_total - G_na / G_total) * woe_i[-1])
#     print(np.array(df_data))
#     print(df_data[0], df_data[1])
#     break
    
    
#     print((df_data[0] - df_data[1]) * woe_i[i])
#     break
#     for i in range(len(woe_i)):
#         iv_i.append((df_data[0] - df_data[1]) * woe_i[i])
    iv_i.append(sum(iv_i))
#     print(iv_i)
#     break
    iv_is.append(iv_i)
    tmp_dict = OrderedDict()
    tmp_dict["name"] = v_name
    tmp_dict["WOE"] = ratio.sum()
    tmp_dict['Type'] = data_type
    WOE_list.append(tmp_dict)

WOE_result = pd.DataFrame(WOE_list)
WOE_result.head()

Float64Index([], dtype='float64', name='v000')
Float64Index([], dtype='float64', name='v001')
Float64Index([], dtype='float64', name='v002')
Float64Index([], dtype='float64', name='v003')
Float64Index([], dtype='float64', name='v004')
Float64Index([], dtype='float64', name='v005')
Float64Index([], dtype='float64', name='v006')
Float64Index([], dtype='float64', name='v007')
Float64Index([], dtype='float64', name='v008')
Float64Index([], dtype='float64', name='v009')
Float64Index([], dtype='float64', name='v010')
Float64Index([], dtype='float64', name='v011')
Float64Index([], dtype='float64', name='v012')
Float64Index([], dtype='float64', name='v013')
Float64Index([], dtype='float64', name='v014')
Float64Index([], dtype='float64', name='v015')
Float64Index([], dtype='float64', name='v016')
Float64Index([], dtype='float64', name='v017')
Float64Index([], dtype='float64', name='v018')
Float64Index([], dtype='float64', name='v019')
Float64Index([], dtype='float64', name='v020')
Float64Index(

Float64Index([], dtype='float64', name='v177')
Float64Index([], dtype='float64', name='v178')
Float64Index([], dtype='float64', name='v179')
Float64Index([], dtype='float64', name='v180')
Float64Index([], dtype='float64', name='v181')
Float64Index([], dtype='float64', name='v182')
Float64Index([], dtype='float64', name='v183')
Float64Index([], dtype='float64', name='v184')
Float64Index([], dtype='float64', name='v185')
Float64Index([], dtype='float64', name='v186')
Float64Index([], dtype='float64', name='v187')
Float64Index([], dtype='float64', name='v188')
Float64Index([], dtype='float64', name='v189')
Float64Index([], dtype='float64', name='v190')
Float64Index([], dtype='float64', name='v191')
Float64Index([], dtype='float64', name='v192')
Float64Index([], dtype='float64', name='v193')
Float64Index([], dtype='float64', name='v194')
Float64Index([], dtype='float64', name='v195')
Float64Index([], dtype='float64', name='v196')
Float64Index([], dtype='float64', name='v197')
Float64Index(

Float64Index([], dtype='float64', name='v355')
Float64Index([], dtype='float64', name='v356')
Float64Index([], dtype='float64', name='v357')
Float64Index([], dtype='float64', name='v358')
Float64Index([], dtype='float64', name='v359')
Float64Index([], dtype='float64', name='v360')
Float64Index([], dtype='float64', name='v361')
Float64Index([], dtype='float64', name='v362')
Float64Index([], dtype='float64', name='v363')
Float64Index([], dtype='float64', name='v364')
Float64Index([], dtype='float64', name='v365')
Float64Index([], dtype='float64', name='v366')
Float64Index([], dtype='float64', name='v367')
Float64Index([], dtype='float64', name='v368')
Float64Index([], dtype='float64', name='v369')
Float64Index([], dtype='float64', name='v370')
Float64Index([], dtype='float64', name='v371')
Float64Index([], dtype='float64', name='v372')
Float64Index([], dtype='float64', name='v373')
Float64Index([], dtype='float64', name='v374')
Float64Index([], dtype='float64', name='v375')
Float64Index(

Float64Index([], dtype='float64', name='v532')
Float64Index([], dtype='float64', name='v533')
Float64Index([], dtype='float64', name='v534')
Float64Index([], dtype='float64', name='v535')
Float64Index([], dtype='float64', name='v536')
Float64Index([], dtype='float64', name='v537')
Float64Index([], dtype='float64', name='v538')
Float64Index([], dtype='float64', name='v539')
Float64Index([], dtype='float64', name='v540')
Float64Index([], dtype='float64', name='v541')
Float64Index([], dtype='float64', name='v542')
Float64Index([], dtype='float64', name='v543')
Float64Index([], dtype='float64', name='v544')
Float64Index([], dtype='float64', name='v545')
Float64Index([], dtype='float64', name='v546')
Float64Index([], dtype='float64', name='v547')
Float64Index([], dtype='float64', name='v548')
Float64Index([], dtype='float64', name='v549')
Float64Index([], dtype='float64', name='v550')
Float64Index([], dtype='float64', name='v551')
Float64Index([], dtype='float64', name='v552')
Float64Index(

Float64Index([], dtype='float64', name='v708')
Float64Index([], dtype='float64', name='v709')
Float64Index([], dtype='float64', name='v710')
Float64Index([], dtype='float64', name='v711')
Float64Index([], dtype='float64', name='v712')
Float64Index([], dtype='float64', name='v713')
Float64Index([], dtype='float64', name='v714')
Float64Index([], dtype='float64', name='v715')
Float64Index([], dtype='float64', name='v716')
Float64Index([], dtype='float64', name='v717')
Float64Index([], dtype='float64', name='v718')
Float64Index([], dtype='float64', name='v719')
Float64Index([], dtype='float64', name='v720')
Float64Index([], dtype='float64', name='v721')
Float64Index([], dtype='float64', name='v722')
Float64Index([], dtype='float64', name='v723')
Float64Index([], dtype='float64', name='v724')
Float64Index([], dtype='float64', name='v725')
Float64Index([], dtype='float64', name='v726')
Float64Index([], dtype='float64', name='v727')
Float64Index([], dtype='float64', name='v728')
Float64Index(

Float64Index([], dtype='float64', name='v886')
Float64Index([], dtype='float64', name='v887')
Float64Index([], dtype='float64', name='v888')
Float64Index([], dtype='float64', name='v889')
Float64Index([], dtype='float64', name='v890')
Float64Index([], dtype='float64', name='v891')
Float64Index([], dtype='float64', name='v892')
Float64Index([], dtype='float64', name='v893')
Float64Index([], dtype='float64', name='v894')
Float64Index([], dtype='float64', name='v895')
Float64Index([], dtype='float64', name='v896')
Float64Index([], dtype='float64', name='v897')
Float64Index([], dtype='float64', name='v898')
Float64Index([], dtype='float64', name='v899')
Float64Index([], dtype='float64', name='v900')
Float64Index([], dtype='float64', name='v901')
Float64Index([], dtype='float64', name='v902')
Float64Index([], dtype='float64', name='v903')
Float64Index([], dtype='float64', name='v904')
Float64Index([], dtype='float64', name='v905')
Float64Index([], dtype='float64', name='v906')
Float64Index(

,name,WOE,Type
0,v000,0.349176,discrete
1,v001,0.064284,continous
2,v002,-0.307234,continous
3,v003,0.560413,continous
4,v004,0.212011,continous


In [25]:
woe_is

[array([0.06977241, 0.27940339, 0.        , 0.        ]),
 array([0.0642837]),
 array([-0.28959287, -0.29220043,  0.32690573, -0.38182619,  0.32947973]),
 array([0.10751372, 0.10747806, 0.11500866, 0.02101769, 0.20939511]),
 array([0.01050698, 0.07816849, 0.11847302, 0.00486246]),
 array([0.02060825, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 array([0.06769036, 0.09309664, 0.14426011, 0.19854584]),
 array([0.1206539]),
 array([ 0.09850168, -0.00353691, -0.0162923 ,  0.23261475,  0.123709  ]),
 array([-0.1839199 ,  0.04372382, -0.03182037,  0.01295216,  0.21778353]),
 array([-0.013892  ,  0.07832191,  0.07926642,  0.07422537]),
 array([ 0.07312206,  0.18003092, -0.70684593,  0.        , -1.3574335 ,
         0.        ,  0.        ,  0.        ,  0.        ]),
 array([0.03753526, 0.00871243

In [ ]:
woe_is[0]

In [ ]:
# np.savetxt("./result/WOEs.txt", woe_is)
f = open("./result/WOEs.txt", 'w')
for i in range(len(woe_is)):
    for j in range(len(woe_is[i])):
        if j == len(woe_is[i]) - 1:
            f.write(str(woe_is[i][j]))
        else:
            f.write(str(woe_is[i][j])+',')
    f.write('\n')
f.close()

In [ ]:
f = open("./result/WOEs.txt", 'w')
f
for i in range(len(woe_is)):
    

In [ ]:
count = 0
for i in range(len(iv_is)):
    if iv_is[i][-1] >= 0.02:
        print(iv_is[i][-1])
        count += 1
    if iv_is[i][-1] > 0.2:
        print(i)
        break
count

In [26]:
iv_is

[[0.0021893504432384026,
  0.00017510642798984936,
  0.0,
  0.0,
  0.0,
  0.002364456871228252],
 [0.0004315132631258261, 0.0004315132631258297, 0.0008630265262516558],
 [0.0005190804758163929,
  0.0005278151650105833,
  0.0004915290205511295,
  0.0009424736544814018,
  0.000499978084282803,
  -0.0010036180551193298,
  0.0019772583450229806],
 [0.0014236059974813268,
  0.001493767696170805,
  0.0016261528336773974,
  5.7863659452510073e-05,
  0.005531818327733628,
  0.014751902786696183,
  0.02488511130121185],
 [1.7549452889996055e-05,
  0.0004736745452111816,
  0.0010622965211219179,
  1.889326359536695e-06,
  8.30753104875664e-05,
  0.0016384851560701987],
 [3.031276833441907e-05,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.031276833441907e-05],
 [0.0012965276938234629,
  0.001595305665884306,
  0.0030468290994460977,
  0.006189086475301833,
  0.01758764373186471,
  0.029715392666320412],
 [

In [ ]:
f = open("./result/IVs.txt", 'w')
for i in range(len(iv_is)):
    for j in range(len(iv_is[i])):
        if j == len(iv_is[i]) - 1:
            f.write(str(iv_is[i][j]))
        else:
            f.write(str(iv_is[i][j])+',')
    f.write('\n')
f.close()

In [27]:
# Compute PSI independently
# 如果是连续性数据，首先排序分为5组，离散化之后，计算WOE
# 如果是离散型数据，直接分类就好（0-10等）, 计算WOE
label = "DEFAULT_LABEL"
n_bins =  5
B_total = df_train[label].sum() # default, y = 1
G_total = len(df_train[label]) - B_total  # not default, y = 0
B_total_test = df_test[label].sum() # default, y = 1
G_total_test = len(df_test[label]) - B_total_test  # not default, y = 0
diverse_threshold = 30

PSIs = []
for v_name in v_names:
    data_type = 'discrete'
    df_v = df_train[[v_name, label]]
    df_v.dropna(inplace = True)
    df_v_test = df_test[[v_name, label]]
    df_v_test.dropna(inplace=True)
    
    KBD = preprocessing.KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')

    if len(df_v[v_name].unique()) > diverse_threshold:
        # continuous 离散化
        # https://scikit-learn.org/stable/modules/preprocessing.html#discretization
        data_type = 'continous'
        data = KBD.fit_transform(df_v[[v_name]])
        df_v[v_name] = data  # 离散化
        data = KBD.transform(df_v_test[[v_name]])
        df_v_test[v_name] = data  # 离散化
    else:
        PSIs.append([v_name, 0.23333333])
        continue
    df_data = pd.pivot_table(df_v, index = v_name,columns=label, values=label, aggfunc=len, fill_value=0)
    df_data[0] = df_data[0] / G_total   # good y=0
    df_data[1] = df_data[1] / B_total   # bad  y=1
    ratio = np.log(df_data[0] / df_data[1])
    ratio.replace([np.inf, -np.inf], 0, inplace=True)
#     print(df_data[1])
    A_is = np.array(df_data[1])
    
    df_data = pd.pivot_table(df_v_test, index = v_name,columns=label, values=label, aggfunc=len, fill_value=0)
    df_data[0] = df_data[0] / G_total   # good y=0
    df_data[1] = df_data[1] / B_total   # bad  y=1
    ratio = np.log(df_data[0] / df_data[1])
    ratio.replace([np.inf, -np.inf], 0, inplace=True)
    E_is = np.array(df_data[1])
#     print(A_is)
#     print(E_is)
    PSI = 0
    for gg in range(len(A_is)):
        if A_is[gg] == E_is[gg]:
            continue
        PSI += (A_is[gg] - E_is[gg]) * math.log(math.e, A_is[gg] / E_is[gg])
#     print(PSI)
    PSIs.append([v_name, PSI])


In [28]:
# Save PSIs into txt file
f = open("./result/PSIs.txt", 'w')
for i in range(len(PSIs)):
    f.write(PSIs[i][0] + ',')
    f.write(str(PSIs[i][1]) + '\n')
f.close()

In [ ]:
pd.isna(train['v003'][4])


In [10]:
# Select intersection of PSI and IV
PSI_select = []
IV_select = []

for i in range(len(PSIs)):
    if PSIs[i][1] <= 0.25:
        PSI_select.append(PSIs[i][0])
for i in range(len(iv_is)):
    if iv_is[i][-1] >= 0.005:
        if i < 10:
            IV_select.append("v00" + str(i))
            continue
        if i < 100:
            IV_select.append("v0" + str(i))
            continue
        if i < 1000:
            IV_select.append("v" + str(i))
shelter = list(set(PSI_select).intersection(set(IV_select)))

In [29]:
train_features = pd.read_csv("./data/train/feature.csv")
train_labels = pd.read_csv("./data/train/label.csv")
train = pd.merge(train_features, train_labels)

X_train = train[shelter]

NameError: name 'shelter' is not defined

In [ ]:
# Delete all features whose nan rate >= 0.8
deleted = 0
all_len = len(X_train)
for col in X_train.columns:
    if 'v' not in col:
        continue
    else:
        if X_train[col].isna().sum() >= 0.9 * all_len:
            X_train.drop(col, axis=1, inplace=True)
            deleted += 1
deleted

In [ ]:
X_train

In [ ]:
f = open("./result/selected_v_1.txt", 'w')
select_cols = X_train.columns.tolist()
for i in range(len(select_cols)):
    f.write(select_cols[i] + '\n')
f.close()

In [ ]:
y_train = train['DEFAULT_LABEL']
y_train

In [4]:
# U(U-shape), P(positve), D(down), N(None)
woes_label = pd.read_csv("./data/woes_labels.csv", names=['Variable', 'Mark'])
woes_label

,Variable,Mark
0,v012,U
1,v017,P
2,v019,N
3,v022,P
4,v030,U
...,...,...
212,v978,N
213,v988,N
214,v989,U
215,v995,U


In [12]:
woe_select = []
for i in range(len(woes_label)):
    if woes_label['Mark'][i] != "N":
        woe_select.append(woes_label['Variable'][i])
len(woe_select)

69

In [7]:
X_train = pd.read_csv("./data/cleaned/X_train.csv")
X_train = X_train.drop("Unnamed: 0", axis=1)
X_train

,v551,v503,v856,v447,v388,v959,v537,v616,v510,v453,...,v989,v033,v294,v275,v953,v176,v925,v381,v577,v678
0,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,0.000000,0.0,0.408661,...,0.0,2.508463,0.211597,0.0,0.0,1.362137,6.732645,0.248366,0.236017,0.522545
1,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
2,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.508463,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
3,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,1.538057,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.831777
4,0.065452,3.834355,0.068335,0.091808,0.085517,0.659167,0.029108,2.280031,0.0,0.201051,...,0.0,2.508463,0.059771,0.0,0.0,1.678527,6.732645,0.395795,0.236017,1.318335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73495,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73496,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73497,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,2.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73498,0.065452,3.834355,0.021539,0.090992,0.033043,1.119132,0.029108,1.039721,0.0,0.178515,...,0.0,2.508463,0.049860,0.0,0.0,1.678527,6.732645,0.248366,0.156252,1.318335


In [14]:
train_select = X_train.columns.tolist()
final_select = list(set(woe_select).intersection(set(train_select)))
len(final_select)

56

In [21]:
f = open("./result/selected_v_f.txt", 'w')
# select_cols = X_train.columns.tolist()
for i in range(len(final_select)):
    f.write(final_select[i] + '\n')
f.close()

In [15]:
X_train = X_train[final_select]
X_train

,v442,v448,v840,v238,v216,v275,v096,v255,v030,v228,...,v350,v616,v540,v498,v916,v100,v687,v156,v017,v033
0,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.447650,0.000000,0.0,0.447650,0.0,0.103457,0.0,0.183050,0.0,2.508463
1,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.183050,0.0,2.375623
2,0.184981,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.000000,0.0,2.508463
3,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.227003,0.0,1.538057
4,0.055824,1.826455,0.046932,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.115187,0.0,0.000000,0.0,2.508463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73495,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.183050,0.0,2.375623
73496,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.183050,0.0,2.375623
73497,0.114239,1.826455,0.101723,0.0,0.0,2.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.183050,0.0,2.375623
73498,0.114239,1.378039,0.023676,0.0,0.0,0.0,0.0,0.0,0.0,0.240805,...,0.021918,1.039721,0.0,0.105668,0.0,0.000000,0.0,0.341140,0.0,2.508463


In [22]:
X_test = pd.read_csv("./data/cleaned/X_test.csv")
X_test = X_test[final_select]
X_test


,v442,v448,v840,v238,v216,v275,v096,v255,v030,v228,...,v350,v616,v540,v498,v916,v100,v687,v156,v017,v033
0,0.114169,1.836756,0.102569,0.0,0.0,1.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.508463
1,0.114169,1.908061,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.660754,...,0.00000,3.119162,0.0,0.00000,0.0,0.102684,0.0,0.147919,0.0,1.940812
2,0.114169,1.836756,0.102569,0.0,0.0,0.0,1.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.212777,0.0,2.508463
3,0.114169,1.836756,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.376681
4,0.114169,1.836756,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.376681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24495,0.114169,1.836756,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.508463
24496,0.114169,1.836756,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.508463
24497,0.147123,1.836756,0.102569,0.0,0.0,1.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.087199,0.0,2.508463
24498,0.082792,1.836756,0.102569,0.0,0.0,1.0,1.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.136072,0.0,2.508463


In [23]:
X_train.to_csv("./data/cleaned/X_train_f.csv")
X_test.to_csv("./data/cleaned/X_test_f.csv")

In [27]:
X_train
target_cols = []
for col in X_train.columns:
    if is_diverse(X_train, col, 30):
        target_cols.append(col)
len(target_cols)

26

In [29]:
f = open("./data/encoded_cols_f.txt", 'w')
# select_cols = X_train.columns.tolist()
for i in range(len(target_cols)):
    f.write(target_cols[i] + '\n')
f.close()

In [26]:
# One-hot encoding and deal with nanx
def is_diverse(data, col, tolerance):
    vc_data = data[col].value_counts()
    if len(vc_data) <= tolerance:
        # first check
#         print(col)
        # vote4integer
        count = 0
        vote = 0
        for i in range(len(data)):
            if not pd.isna(data[col][i]):
                count += 1
                if int(data[col][i]) == data[col][i]:
                    vote += 1
        if vote == count:
            return True
    return False

In [ ]:
target_cols = []
for col in X_train.columns:
    if is_diverse(X_train, col, 30):
        target_cols.append(col)

In [ ]:
len(target_cols)

In [ ]:
X_train

In [ ]:
# df = X_train.copy(deep=True)
for column in list(X_train.columns[X_train.isnull().sum() > 0]):
    if column in target_cols:
        middle_val = X_train[column].median()
        X_train[column].fillna(middle_val, inplace=True)
        continue
#     print(column)
#     break
    mean_val = X_train[column].mean()
    X_train[column].fillna(mean_val, inplace=True)
# X_train

In [27]:
X_train.to_csv("./data/cleaned/X_train.csv")
y_train.to_csv("./data/cleaned/y_train.csv")

In [ ]:
# Apply cleaning also towards test
test_features = pd.read_csv("./data/test/feature.csv")
test_labels = pd.read_csv("./data/test/label.csv")
test = pd.merge(test_features, test_labels)

X_test = test[X_train.columns.to_list()]
y_test = test['DEFAULT_LABEL']

In [ ]:
X_test

In [ ]:
for column in list(X_test.columns[X_test.isnull().sum() > 0]):
    if column in target_cols:
        middle_val = X_test[column].median()
        X_test[column].fillna(middle_val, inplace=True)     # median value for diverse variable
        continue
    mean_val = X_test[column].mean()
    X_test[column].fillna(mean_val, inplace=True)
X_test

In [31]:
X_test.to_csv("./data/cleaned/X_test.csv")
y_test.to_csv("./data/cleaned/y_test.csv")

In [32]:
# Try to onehot encoding
target_cols_after_fillna = []
for col in X_train.columns:
    if is_diverse(X_train, col, 30):
        target_cols_after_fillna.append(col)

In [33]:
len(target_cols_after_fillna)

62

In [2]:
# Try to onehot encoding
X_train = pd.read_csv("./data/cleaned/X_train.csv")
X_train = X_train.drop("Unnamed: 0", axis=1)
X_train

,v551,v503,v856,v447,v388,v959,v537,v616,v510,v453,...,v989,v033,v294,v275,v953,v176,v925,v381,v577,v678
0,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,0.000000,0.0,0.408661,...,0.0,2.508463,0.211597,0.0,0.0,1.362137,6.732645,0.248366,0.236017,0.522545
1,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
2,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.508463,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
3,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,1.538057,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.831777
4,0.065452,3.834355,0.068335,0.091808,0.085517,0.659167,0.029108,2.280031,0.0,0.201051,...,0.0,2.508463,0.059771,0.0,0.0,1.678527,6.732645,0.395795,0.236017,1.318335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73495,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73496,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73497,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,2.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73498,0.065452,3.834355,0.021539,0.090992,0.033043,1.119132,0.029108,1.039721,0.0,0.178515,...,0.0,2.508463,0.049860,0.0,0.0,1.678527,6.732645,0.248366,0.156252,1.318335


In [5]:
target_cols = []
for col in X_train.columns:
    if is_diverse(X_train, col, 30):
        target_cols.append(col)

In [21]:
f = open('./data/encoded_cols.txt', 'w')
for i in range(len(target_cols)):
    f.write(target_cols[i] + '\n')
f.close()
target_cols

['v510',
 'v048',
 'v827',
 'v505',
 'v912',
 'v271',
 'v669',
 'v366',
 'v410',
 'v306',
 'v896',
 'v490',
 'v890',
 'v036',
 'v372',
 'v770',
 'v334',
 'v096',
 'v216',
 'v157',
 'v691',
 'v110',
 'v747',
 'v632',
 'v997',
 'v916',
 'v243',
 'v146',
 'v550',
 'v697',
 'v564',
 'v160',
 'v066',
 'v576',
 'v424',
 'v930',
 'v791',
 'v255',
 'v071',
 'v687',
 'v877',
 'v813',
 'v077',
 'v030',
 'v053',
 'v742',
 'v456',
 'v241',
 'v617',
 'v595',
 'v728',
 'v540',
 'v147',
 'v165',
 'v017',
 'v238',
 'v964',
 'v590',
 'v212',
 'v989',
 'v275',
 'v953']

In [8]:
X_train['v510']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
73495    0.0
73496    0.0
73497    0.0
73498    0.0
73499    0.0
Name: v510, Length: 73500, dtype: float64

In [9]:
from sklearn.preprocessing import LabelBinarizer

In [29]:
test = pd.DataFrame({'data1': [1, 2, 2, 3], 'data2': [2, 3, 4, 5]})
col = 'v510'
LB = LabelBinarizer()
y = LB.fit_transform(test['data1'])

In [39]:
y = y.tolist()

In [41]:
test['data1'] = y
test

,data1,data2,data1_ohe
0,"[1, 0, 0]",2,"[1, 0, 0]"
1,"[0, 1, 0]",3,"[0, 1, 0]"
2,"[0, 1, 0]",4,"[0, 1, 0]"
3,"[0, 0, 1]",5,"[0, 0, 1]"


In [12]:
X_train['v510'].value_counts()

0.0     59973
1.0      9051
2.0      2958
3.0       963
4.0       365
5.0       116
6.0        39
7.0        15
8.0        13
11.0        2
9.0         2
15.0        1
12.0        1
10.0        1
Name: v510, dtype: int64

In [43]:
# Start to transform
LB_classes = []
for col in target_cols:
    LB = LabelBinarizer()
    y = LB.fit_transform(X_train[col])
    single_class = LB.classes_
    LB_classes.append([col, single_class])
    X_train[col] = y.tolist()
    
    y = LB.transform(X_test[col])
    
    
X_train

,v551,v503,v856,v447,v388,v959,v537,v616,v510,v453,...,v989,v033,v294,v275,v953,v176,v925,v381,v577,v678
0,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,0.000000,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.408661,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2.508463,0.211597,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.362137,6.732645,0.248366,0.236017,0.522545
1,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.245950,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2.375623,0.059771,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.289099,6.732645,0.248366,0.236017,0.522545
2,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.245950,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2.508463,0.059771,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.289099,6.732645,0.248366,0.236017,0.522545
3,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.245950,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",1.538057,0.059771,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.289099,6.732645,0.248366,0.236017,0.831777
4,0.065452,3.834355,0.068335,0.091808,0.085517,0.659167,0.029108,2.280031,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.201051,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2.508463,0.059771,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.678527,6.732645,0.395795,0.236017,1.318335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73495,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.245950,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2.375623,0.059771,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.289099,6.732645,0.248366,0.236017,0.522545
73496,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.245950,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2.375623,0.059771,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.289099,6.732645,0.248366,0.236017,0.522545
73497,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.245950,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2.375623,0.059771,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.289099,6.732645,0.248366,0.236017,0.522545
73498,0.065452,3.834355,0.021539,0.090992,0.033043,1.119132,0.029108,1.039721,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.178515,...,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",2.508463,0.049860,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]",1.678527,6.732645,0.248366,0.156252,1.318335


In [ ]:
# Smote algorithm
from imblearn.over_sampling import SMOTE
smo = SMOTE(ratio={1:15000},random_state=42)

X_smo, y_smo = smo.fit_sample(X_train, y_train)

In [ ]:
# # 使用sklearn的make_classification生成不平衡数据样本
# from sklearn.datasets import make_classification
# from imblearn.over_sampling import SMOTE
# # 生成一组0和1比例为9比1的样本，X为特征，y为对应的标签
# X, y = make_classification(n_classes=2, class_sep=2,
#                            weights=[0.9, 0.1], n_informative=3, 
#                            n_redundant=1, flip_y=0,
#                            n_features=20, n_clusters_per_class=1, 
#                            n_samples=1000, random_state=10)
# from collections import Counter
# # 查看所生成的样本类别分布，0和1样本比例9比1，属于类别不平衡数据
# print(Counter(y))
# # Counter({0: 900, 1: 100})

# smo = SMOTE(random_state=42)

# X_smo, y_smo = smo.fit_sample(X, y)
# print(Counter(y_smo))

In [ ]:
# Data process
# Look through data: diverse or continuous. One-hot encoding or not
# Smote algorithm
# Train and Validation

In [ ]:
# Model selection
# Linear regression, RandomForest, SVM
# XGBoost, LightGBM

In [35]:
X_train = pd.read_csv("./data/cleaned/X_train_f.csv")
X_train.drop("Unnamed: 0", axis=1, inplace=True)
X_train

,v442,v448,v840,v238,v216,v275,v096,v255,v030,v228,...,v350,v616,v540,v498,v916,v100,v687,v156,v017,v033
0,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.447650,0.000000,0.0,0.447650,0.0,0.103457,0.0,0.183050,0.0,2.508463
1,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.183050,0.0,2.375623
2,0.184981,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.000000,0.0,2.508463
3,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.227003,0.0,1.538057
4,0.055824,1.826455,0.046932,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.115187,0.0,0.000000,0.0,2.508463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73495,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.183050,0.0,2.375623
73496,0.114239,1.826455,0.101723,0.0,0.0,0.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.183050,0.0,2.375623
73497,0.114239,1.826455,0.101723,0.0,0.0,2.0,0.0,0.0,0.0,0.344942,...,0.091325,2.280031,0.0,0.120078,0.0,0.103457,0.0,0.183050,0.0,2.375623
73498,0.114239,1.378039,0.023676,0.0,0.0,0.0,0.0,0.0,0.0,0.240805,...,0.021918,1.039721,0.0,0.105668,0.0,0.000000,0.0,0.341140,0.0,2.508463


In [36]:
# PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
pca.fit(X_train)

PCA(n_components=10)

In [37]:
pca.components_

array([[-1.10216745e-03,  1.21747713e-02, -2.01283965e-04,
         2.37968800e-02,  1.17641076e-02, -7.69547848e-03,
         6.99234895e-03,  8.13436957e-04,  7.66269108e-04,
         2.95426553e-03, -1.09335861e-03,  8.54410669e-03,
        -5.86281170e-04, -2.26746386e-04,  1.75684464e-04,
        -8.10820087e-05,  1.11141454e-03, -1.52154055e-03,
         2.35037016e-03, -3.37621011e-04,  1.13476864e-03,
         9.97412594e-01, -2.23060002e-03,  5.03108373e-04,
        -1.46098536e-04,  6.03330071e-04,  9.47186837e-04,
         1.02189661e-03,  2.39451982e-02,  1.02087385e-03,
        -2.74300878e-04,  3.06602358e-02,  2.77194390e-02,
        -3.97602432e-04,  5.32091847e-03,  1.22658951e-02,
        -1.19344561e-02,  1.84473430e-04, -1.87772235e-04,
        -9.00665517e-04,  1.93747610e-04,  2.62922436e-03,
        -1.38852592e-03,  2.48700312e-03,  9.91032639e-04,
         6.61350822e-03, -1.39766458e-03,  3.63518810e-02,
         1.83412294e-04, -1.70627421e-03,  9.74515103e-0

In [42]:
X_train_pca = pd.DataFrame(pca.transform(X_train), columns=['v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9'])

In [43]:
X_train_pca

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9
0,-0.649643,-2.755261,0.352282,-0.730618,0.063895,-0.051412,-0.230647,-0.161060,0.200213,-0.100509
1,-0.515240,0.077930,0.080118,-0.486040,0.003633,-0.037138,-0.012396,-0.162855,0.189151,-0.064697
2,0.470512,-0.011480,0.016195,-0.186230,-0.034279,-0.069804,0.030693,-0.237166,-0.533294,0.509748
3,-0.525754,0.005040,0.010357,-0.079776,-0.011857,0.016250,0.014598,-0.159004,-0.536218,0.308353
4,-0.516693,-0.023183,-2.088216,-0.814384,0.057166,-0.092294,-1.016914,-0.161427,0.218983,-0.014632
...,...,...,...,...,...,...,...,...,...,...
73495,-0.517471,0.066121,0.064307,-0.410589,0.000991,0.019473,-0.009660,-0.151592,0.211834,-0.083231
73496,-0.515240,0.077930,0.080118,-0.486040,0.003633,-0.037138,-0.012396,-0.162855,0.189151,-0.064697
73497,-1.541370,-0.017969,-0.138333,0.774143,0.013166,1.658795,-0.039371,-0.187765,-0.531200,0.397083
73498,-1.563132,-1.337095,0.384632,-0.557192,0.087562,-0.063143,-0.906751,-0.143137,0.204625,-0.090264


In [44]:
X_test = pd.read_csv("./data/cleaned/X_test_f.csv")
X_test.drop("Unnamed: 0", axis=1, inplace=True)
X_test

,v442,v448,v840,v238,v216,v275,v096,v255,v030,v228,...,v350,v616,v540,v498,v916,v100,v687,v156,v017,v033
0,0.114169,1.836756,0.102569,0.0,0.0,1.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.508463
1,0.114169,1.908061,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.660754,...,0.00000,3.119162,0.0,0.00000,0.0,0.102684,0.0,0.147919,0.0,1.940812
2,0.114169,1.836756,0.102569,0.0,0.0,0.0,1.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.212777,0.0,2.508463
3,0.114169,1.836756,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.376681
4,0.114169,1.836756,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.376681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24495,0.114169,1.836756,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.508463
24496,0.114169,1.836756,0.102569,0.0,0.0,0.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.182618,0.0,2.508463
24497,0.147123,1.836756,0.102569,0.0,0.0,1.0,0.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.087199,0.0,2.508463
24498,0.082792,1.836756,0.102569,0.0,0.0,1.0,1.0,0.0,0.0,0.339662,...,0.09156,2.292626,0.0,0.12122,0.0,0.102684,0.0,0.136072,0.0,2.508463


In [45]:
X_test_pca = pd.DataFrame(pca.transform(X_test), columns=['v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9'])
X_test_pca

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9
0,-0.521378,0.052937,0.081593,-0.085706,0.002929,0.870334,-0.021699,-0.149773,0.285341,-0.076793
1,2.548880,1.187458,0.262731,-0.355067,-0.088502,-0.013984,-0.393665,-0.204342,0.205809,0.015865
2,-0.477650,-0.108092,-0.330610,0.646021,-0.017942,-0.279231,0.631003,-0.245852,-1.503131,-0.584280
3,-0.511936,0.082835,0.137834,-0.388928,-0.000559,0.030002,-0.017683,-0.132438,0.160007,-0.168729
4,-0.514205,0.091988,0.152456,-0.473724,0.002228,-0.035730,0.001588,-0.163087,0.188236,-0.065435
...,...,...,...,...,...,...,...,...,...,...
24495,-1.513048,0.133974,0.158088,-0.461835,0.028736,-0.042976,-0.016362,-0.152793,0.181227,-0.074442
24496,0.490536,0.072747,0.234945,-0.457535,-0.019210,-0.029982,-0.201611,-0.177271,0.188102,-0.045247
24497,-1.563032,-0.063320,-0.493192,0.112481,-0.011846,0.837173,1.198951,-0.185490,-0.445087,0.409064
24498,-0.511993,0.002821,-0.020893,0.545675,-0.015946,0.690837,-0.049499,0.777214,-0.901781,-0.313446


In [46]:
X_train_pca.to_csv("./data/cleaned/X_train_pca.csv")
X_test_pca.to_csv("./data/cleaned/X_test_pca.csv")

In [47]:
train_features = pd.read_csv("./data/train/feature.csv")
train_labels = pd.read_csv("./data/train/label.csv")

test_labels = pd.read_csv("./data/test/label.csv")

In [48]:
test_labels

,APPLICATION_ID,APPLICATION_DATE,DEFAULT_LABEL
0,8f14d6843d638c949859518f53983344,2021-07-25,0
1,5368e8b191c49099c0d1b619060e34bf,2021-08-16,0
2,0588603914b15144d3948e5c965b4a0a,2021-08-04,0
3,948814dc66bfcfc945fe06919f3154fd,2021-07-10,0
4,c64b1041dd80363935a55a9da300e46f,2021-05-23,0
...,...,...,...
24495,8180ee8f1d49944013c9e38e411e6a84,2021-06-04,0
24496,5fc35d484b4935cc56541be1a10535a9,2021-06-15,0
24497,346c1bf3f4b91063ea045ee949b34444,2021-06-25,0
24498,008a4eb9969c9f610c3b4d99c68969a9,2021-06-26,0


In [49]:
list(set(train_labels['APPLICATION_ID']).intersection(set(test_labels['APPLICATION_ID'])))

[]

In [50]:
train_labels['APPLICATION_DATE'].value_counts()

2021-08-29    2104
2021-06-28    1593
2021-06-10    1508
2021-07-05    1201
2021-08-16    1198
              ... 
2021-09-12     321
2021-08-25     320
2021-09-01     319
2021-08-31     313
2021-08-10     309
Name: APPLICATION_DATE, Length: 115, dtype: int64

In [51]:
test_labels['APPLICATION_DATE'].value_counts()

2021-08-29    650
2021-06-28    546
2021-06-10    517
2021-08-16    426
2021-07-05    413
             ... 
2021-09-11    106
2021-09-01    105
2021-08-24    101
2021-09-18    101
2021-08-10     91
Name: APPLICATION_DATE, Length: 115, dtype: int64

In [53]:
max(train_labels['APPLICATION_DATE']), min(train_labels['APPLICATION_DATE'])

('2021-09-18', '2021-05-20')

In [54]:
max(test_labels['APPLICATION_DATE']), min(test_labels['APPLICATION_DATE'])

('2021-09-18', '2021-05-20')

In [56]:
# Construct Table 2 
# Var, max, min, avg, fangcha, 25%, 50%, 75%, PSI, WOE, IV, mean in default, in normal, variance in default, in normal

t2_df = pd.read_csv("./data/cleaned/X_train.csv")
t2_df.drop("Unnamed: 0", axis=1, inplace=True)
t2_df

,v551,v503,v856,v447,v388,v959,v537,v616,v510,v453,...,v989,v033,v294,v275,v953,v176,v925,v381,v577,v678
0,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,0.000000,0.0,0.408661,...,0.0,2.508463,0.211597,0.0,0.0,1.362137,6.732645,0.248366,0.236017,0.522545
1,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
2,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.508463,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
3,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,1.538057,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.831777
4,0.065452,3.834355,0.068335,0.091808,0.085517,0.659167,0.029108,2.280031,0.0,0.201051,...,0.0,2.508463,0.059771,0.0,0.0,1.678527,6.732645,0.395795,0.236017,1.318335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73495,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73496,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,0.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73497,0.065452,3.834355,0.068335,0.090992,0.033043,1.119132,0.029108,2.280031,0.0,0.245950,...,0.0,2.375623,0.059771,2.0,0.0,1.289099,6.732645,0.248366,0.236017,0.522545
73498,0.065452,3.834355,0.021539,0.090992,0.033043,1.119132,0.029108,1.039721,0.0,0.178515,...,0.0,2.508463,0.049860,0.0,0.0,1.678527,6.732645,0.248366,0.156252,1.318335


In [ ]:
construction = pd.Dataframe(columns=['Vname', 'max', 'min', 'avg', 'Variance', '25%', '50%', '75%', 
                                     'PSI', 'WOE', 'IV', 'mean in default', 'mean in normal', 
                                     'variance in default', 'variance in normal'])
construction['Vname'] = t2_df.columns.to

In [57]:
sample_sub = pd.read_csv("./data/sample_submission.csv")
sample_sub

,APPLICATION_ID,APPLICATION_DATE,0,1
0,e390d2817c393edd7d2ffb9f0938399275b1f135,2021-06-28,0.362147,0.637853
1,99b20a430100265d5668e659e0fa8b994c2f3b0e,2021-08-23,0.415639,0.584361
2,4305a9e16c17a0245bb3af1498805d1451d523b8,2021-09-17,0.149944,0.850056
3,ed0570d18e99e49d338b4510518aeb5aeb97b026,2021-06-23,0.442165,0.557835
4,c926e324c5b2f764608991638d7986ab9e2e5c5d,2021-06-09,0.276872,0.723128
...,...,...,...,...
2995,eb5481ac1f703dc860b77a86356ffeae90598a84,2021-08-18,0.646260,0.353740
2996,3d80542dce12ba1be6c91348b3bad00d8f4c8300,2021-07-09,0.990167,0.009833
2997,102565e034fbd8f93c696162e14f5ba1b32694f0,2021-08-14,0.228941,0.771059
2998,ac7b4b9a89e5128d8ec6863f35c6bb11d68c2f94,2021-09-02,0.538027,0.461973


In [67]:
X_test_df = pd.read_csv("./data/test/feature.csv")
X_test_df

,APPLICATION_ID,v000,v001,v002,v003,v004,v005,v006,v007,v008,...,v990,v991,v992,v993,v994,v995,v996,v997,v998,v999
0,8f14d6843d638c949859518f53983344,0.0,NaN,NaN,5.441916,NaN,NaN,5.0,0.0,1.039721,...,8.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,5368e8b191c49099c0d1b619060e34bf,NaN,NaN,NaN,4.969729,0.226694,NaN,NaN,NaN,0.433145,...,NaN,0.175,NaN,NaN,NaN,0.029064,NaN,NaN,NaN,NaN
2,0588603914b15144d3948e5c965b4a0a,0.0,NaN,NaN,4.594687,0.628390,NaN,0.0,0.0,1.008882,...,0.0,NaN,NaN,NaN,NaN,0.006942,15.0,1.0,NaN,0.0
3,948814dc66bfcfc945fe06919f3154fd,0.0,NaN,NaN,NaN,NaN,NaN,20.0,0.0,NaN,...,25.0,NaN,NaN,NaN,NaN,NaN,41.0,0.0,NaN,0.0
4,c64b1041dd80363935a55a9da300e46f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.039721,...,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24495,8180ee8f1d49944013c9e38e411e6a84,NaN,NaN,NaN,2.078853,NaN,NaN,13.0,0.0,0.998622,...,321.0,0.000,NaN,NaN,NaN,NaN,324.0,0.0,NaN,0.0
24496,5fc35d484b4935cc56541be1a10535a9,NaN,NaN,NaN,2.004421,NaN,NaN,0.0,0.0,0.254696,...,NaN,0.000,NaN,NaN,NaN,0.051186,3.0,0.0,NaN,0.0
24497,346c1bf3f4b91063ea045ee949b34444,0.0,NaN,NaN,5.405959,0.000000,NaN,6.0,0.0,0.649555,...,0.0,0.000,NaN,NaN,NaN,0.001088,8.0,1.0,0.831777,0.0
24498,008a4eb9969c9f610c3b4d99c68969a9,0.0,NaN,NaN,5.528423,0.000000,NaN,4.0,0.0,1.013514,...,0.0,0.000,NaN,NaN,NaN,NaN,51.0,0.0,0.831777,0.0


In [68]:
list(set(sample_sub['APPLICATION_ID']).intersection(set(X_test_df['APPLICATION_ID'])))

[]

In [63]:
# Test with sample_sub
labels = []
for i in range(len(3000)):
    if sample_sub['0'][i] > sample_sub['1'][i]:
        labels.append()
    else 1

sample_sub['label'] = [ 0 ]

SyntaxError: invalid syntax (804583379.py, line 2)

In [59]:
y_test = pd.read_csv("./data/cleaned/y_test.csv")
y_test.drop("Unnamed: 0", axis=1, inplace=True)
y_test

,DEFAULT_LABEL
0,0
1,0
2,0
3,0
4,0
...,...
24495,0
24496,0
24497,0
24498,0


In [66]:
X_test = pd.read_csv("./data/cleaned/X_test.csv")
X_test.drop("Unnamed: 0", axis=1, inplace=True)
X_test

,v551,v503,v856,v447,v388,v959,v537,v616,v510,v453,...,v989,v033,v294,v275,v953,v176,v925,v381,v577,v678
0,0.065219,3.857375,0.073422,0.089633,0.033283,1.130778,0.030061,2.292626,0.0,0.248302,...,0.0,2.508463,0.061294,1.0,0.0,1.296386,6.720439,0.248882,0.243797,0.528724
1,0.065219,3.857375,0.004069,0.089633,0.033283,1.130778,0.030061,3.119162,0.0,0.324372,...,0.0,1.940812,0.062389,0.0,0.0,1.678527,6.720439,0.248882,0.115687,0.000000
2,0.065219,3.857375,0.008400,0.089633,0.033283,1.130778,0.030061,2.292626,0.0,0.269178,...,0.0,2.508463,0.061294,0.0,0.0,1.296386,6.720439,0.248882,0.243797,0.528724
3,0.065219,3.857375,0.073422,0.089633,0.033283,1.130778,0.030061,2.292626,0.0,0.248302,...,0.0,2.376681,0.061294,0.0,1.0,1.296386,6.720439,0.248882,0.243797,0.528724
4,0.065219,3.857375,0.073422,0.089633,0.033283,1.130778,0.030061,2.292626,0.0,0.248302,...,0.0,2.376681,0.061294,0.0,0.0,1.296386,6.720439,0.248882,0.243797,0.528724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24495,0.065219,3.857375,0.073422,0.089633,0.033283,1.130778,0.030061,2.292626,0.0,0.230145,...,0.0,2.508463,0.061294,0.0,0.0,0.970406,6.720439,0.248882,0.243797,0.528724
24496,0.065219,3.857375,0.120581,0.089633,0.033283,1.130778,0.030061,2.292626,0.0,0.230145,...,0.0,2.508463,0.061294,0.0,0.0,0.970406,6.720439,0.248882,0.243797,0.831777
24497,0.065219,3.857375,0.073422,0.021506,0.000263,1.130778,0.030061,2.292626,0.0,0.269178,...,0.0,2.508463,0.061294,1.0,0.0,0.485203,6.720439,0.248882,0.243797,0.528724
24498,0.065219,3.857375,0.073422,0.042559,0.000430,1.130778,0.030061,2.292626,0.0,0.178515,...,0.0,2.508463,0.061294,1.0,0.0,1.126607,6.720439,0.248882,0.243797,1.318335


In [60]:
y_test['DEFAULT_LABEL'].value_counts()

0    23986
1      514
Name: DEFAULT_LABEL, dtype: int64

In [62]:
y_train['DEFAULT_LABEL'].value_counts()

0    71957
1     1543
Name: DEFAULT_LABEL, dtype: int64